In [1]:
import cv2 as cv
import numpy as np

In [2]:
width=320
confidenceLvl = 0.7
nms_threshold = 0.3

In [3]:
with open('cocolabels.txt','rt') as f:
    classNames = f.read()
    f.close()
classNames = classNames.split('\n')
cfgFile = 'yolov3.cfg'#network architecture
netWeight = 'yolov3.weights' # weight of network
yoloNet = cv.dnn.readNetFromDarknet(cfgFile,netWeight)
yoloNet.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
yoloNet.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)

In [4]:
def findObj(outputs,img):
    hT,wT,cT = img.shape
    bndingB = []
    classIds =[]
    confs = []
    
    for output in outputs:
        for det in output:
            scores = det[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            
            if confidence > confidenceLvl:
                w,h = int(det[2]*wT),int(det[3]*hT)
                x,y = int(det[0]*wT-w/2),int(det[1]*hT-h/2)
                bndingB.append([x,y,w,h])
                classIds.append(classId)
                confs.append(float(confidence))
    indices = cv.dnn.NMSBoxes(bndingB,confs,confidenceLvl,nms_threshold=nms_threshold)
    
    for i in indices:
        i = i[0]
        box = bndingB[i]
        x,y,w,h=box[0],box[1],box[2],box[3]
        cv.rectangle(img,(x,y),(x+w,y+h),(255,0,255),2)
        cv.putText(img,f'{classNames[classIds[i]].upper()}{int(confs[i]*100)}%',(x,y-10),
                   cv.FONT_HERSHEY_SIMPLEX,0.6,(255,0,255),2)
    

In [5]:
cap = cv.VideoCapture(0)
while True:
    _,img = cap.read()
    img = cv.flip(img,1)
    blob = cv.dnn.blobFromImage(img,1/255,(width,width),[0,0,0],1,crop=False)
    yoloNet.setInput(blob)
    
    layerNames = yoloNet.getLayerNames()
    outLayerName = yoloNet.getUnconnectedOutLayersNames()
    
    outputs = yoloNet.forward(outLayerName)
    findObj(outputs,img)
    cv.imshow('image',img)
    if cv.waitKey(1) & 0xff == ord('q'):
        break
cap.release()
cv.destroyAllWindows()